## Inserting data into people and role junction tabels

In [28]:
import pickle
import psycopg2
import regex as re
import pandas as pd
import os.path as path

#### Importing the data

In [2]:
def pickler(file_name):	
    parent = path.abspath(path.join("people_and_roles_tabel_insert.ipynb", "../../.."))
    file_path = "/Data/Extracted/"
    with open(f'{parent + file_path + file_name}.obj', 'rb') as f:
        return pickle.load(f)

In [3]:
j = "junction_table"
df_people = pickler("df_people")
df_daglig_leder = pickler(f"df_daglig_leder{j}")
df_innehaver = pickler(f"df_innehaver{j}")
df_kontaktperson = pickler(f"df_kontaktperson{j}")
df_nestleder = pickler(f"df_nestleder{j}")
df_styremedlem = pickler(f"df_styremedlem{j}")
df_styrets_leder = pickler(f"df_styrets_leder{j}")
df_varamedlem = pickler(f"df_varamedlem{j}")

##### Database connection

In [22]:
def open_connection():
    """Opens a connection to the Azure database

    Returns:
        connection: returns the connection to the database
    """
    conn = psycopg2.connect(database='pgsql-graduate-gruppe3',
                            user='snartimaal',
                            password='#st44p44nu',
                            host='pg-graduate-delt.postgres.database.azure.com')
    return conn

##### Inserting data into the people database

In [5]:
# First checking for ' key

for index, row in df_people.iterrows():
    for char in row['navn']:
        if char == "'":
            print("FUCK")
            print(row['navn'])

FUCK
Cathal Patrick John O'gorman


In [6]:
def format_names(name):

    if re.search("[a-z]'[a-z]", name, re.IGNORECASE):
        formated_name = ""
        for char in name:
            if char == "'":
                letter = "''"
            else:
                letter = char
            formated_name = formated_name + letter
        return formated_name
    else:
        return name

In [7]:
df_people['navn'] = df_people['navn'].map(format_names)

In [8]:
# Adding the people to the people table
try:
    conn = open_connection()
    cur = conn.cursor()

    for index, row in df_people.iterrows():
        cur.execute(f'''
        INSERT INTO people (navn)
        VALUES ('{row['navn']}') ;''')
    
    print("Success")
    conn.commit()
except:
    print("Failure")

finally:
    cur.close()
    conn.close()

Success


#### Inserting into the junction tabels

In [9]:
df_daglig_leder.head(1)

,organisasjonsnummer,person_id,Fratrådt
0,927526891,350,False


In [10]:
def insert_into_junction_table(df, tabel_name):
    """Function that inserts data into the role junction tabels.

    Args:
        df (dataframe): a pandas dataframe
        tabel_name (str): the nave of the target tabel
    """
    try:
        conn = open_connection()
        cur = conn.cursor()

        for index, row in df.iterrows():
            cur.execute(f'''
            INSERT INTO {tabel_name}_junction ( organisasjonsnummer, 
                                                person_id,
                                                fratrådt)
                                                
            VALUES ( {int(row['organisasjonsnummer'])},
                     {int(row['person_id']+1)} ,
                     {int(row['Fratrådt'])} ) ;''')
        
        print("Success")
        conn.commit()
    except:
        print("Failure")

    finally:
        cur.close()
        conn.close()

In [11]:
name_list = [   "daglig_leder", 
                "innehaver", 
                "kontaktperson", 
                "nestleder", 
                "styremedlem",
                "styrets_leder",
                "varamedlem"]

df_j = [        df_daglig_leder,
                df_innehaver,
                df_kontaktperson,
                df_nestleder,
                df_styremedlem,
                df_styrets_leder,
                df_varamedlem  ]

In [12]:
# Changing bool to 0 or 1:
# False == 0, and True == 1:
def bool_to_num(bool):
    if bool == True:
        return 1
    else:
        return 0

for df in df_j:
    df['Fratrådt'] = df['Fratrådt'].map(bool_to_num)

In [13]:
for name, df in zip(name_list, df_j):
    insert_into_junction_table(df, name)

Success
Success
Success
Success
Success
Success
Success


##### Adding key links between tabels

In [30]:
def fk_link(fk_to, fk_from, column_1, column_2):
    """Function that adds forgein key constraints to the fk_to tabel, referencing fk_from.

    Args:
        fk_from (str): name of the tabel getting refferenced
        fk_to (str): name of the tabel getting the constraint
        column (str): name of the column in question
    """
    try:
        conn = open_connection()
        cur = conn.cursor()

        cur.execute(f'''

        ALTER TABLE {fk_to}
        ADD FOREIGN KEY ({column_1}) REFERENCES {fk_from}({column_2});
        
        ''')
        
        print("Success")
        conn.commit()
    except:
        print("Failure")

    finally:
        cur.close()
        conn.close()

In [27]:
# Adding the people person_id as FK in the _junction tables
suffix = "_junction"

for name in name_list:
    fk_link(name+suffix, "people", "person_id", "person_id")
    print(name+suffix)

Success
daglig_leder_junction
Success
innehaver_junction
Success
kontaktperson_junction
Success
nestleder_junction
Success
styremedlem_junction
Success
styrets_leder_junction
Success
varamedlem_junction


In [31]:
# Adding the orgnumber_junction organisasjonsnummer as FK in the _junction tables
suffix = "_junction"

for name in name_list:
    fk_link(name+suffix, "orgnumber_junction", "organisasjonsnummer", "orgnumber")
    print(name+suffix)

Success
daglig_leder_junction
Success
innehaver_junction
Success
kontaktperson_junction
Success
nestleder_junction
Success
styremedlem_junction
Success
styrets_leder_junction
Success
varamedlem_junction
